In [38]:
from numpy import mean
from numpy import std
from sklearn.svm import SVC
from sklearn.ensemble import StackingClassifier
from matplotlib import pyplot
from sklearn.tree import DecisionTreeClassifier

In [39]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [40]:
from google.colab import drive
drive.mount('/content/gdrive')

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [41]:
Abhi = pd.read_csv("/content/gdrive/MyDrive/Datasets/mina data 2.0.7-2.0.8.csv")

In [42]:
Abhi.head(50)

,CBO,NOC,WMC,RFC,SLOC,DIT,LCOM,change
0,1,0,4,53,15,0,100,no
1,10,0,12,391,74,2,66,no
2,25,2,181,194,225,0,98,no
3,4,0,1,14,47,0,66,no
4,34,0,40,53,1049,0,0,no
5,0,0,0,13,2,0,0,no
6,23,1,176,370,800,1,0,no
7,3,0,4,17,66,0,0,no
8,5,0,11,390,46,2,9,no
9,7,0,7,20,86,0,0,no


In [43]:
Abhi['change'] = Abhi['change'].map({'yes': 1, 'no': 0})

In [44]:
Abhi.shape

(610, 8)

In [45]:
Abhi.head()

,CBO,NOC,WMC,RFC,SLOC,DIT,LCOM,change
0,1,0,4,53,15,0,100,0
1,10,0,12,391,74,2,66,0
2,25,2,181,194,225,0,98,0
3,4,0,1,14,47,0,66,0
4,34,0,40,53,1049,0,0,0


In [46]:
X = Abhi.drop('change', axis=1)
y = Abhi['change']

In [47]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20)

In [48]:
def get_stacking():
	# define the base models
	level0 = list()
	level0.append(('sig', SVC(probability=True,kernel='sigmoid')))
	level0.append(('lin', SVC(probability=True,kernel='linear')))
	level0.append(('poly', SVC(probability=True,kernel='poly')))
	level0.append(('rbf', SVC(probability=True,kernel='rbf')))
	# define meta learner model
	level1 = DecisionTreeClassifier()
  #level1 = DecisionTreeClassifier()
	# define the stacking ensemble
	model = StackingClassifier(estimators=level0, final_estimator=level1)
	return model

In [49]:
def get_models():
	models = dict()
	#models['lin'] = SVC(kernel='linear')
	#models['sig'] = SVC(kernel='sigmoid')
	#models['poly'] = SVC(kernel='poly')
	#models['rbf'] = SVC(kernel='rbf')

	models['stacking'] = get_stacking()
	return models

In [50]:
# evaluate a give model
def evaluate_model(model, X, y):
  model=model.fit(X_train,y_train)
  y_pred = model.predict(X_test)
  return y_pred

In [51]:
models = get_models()

In [52]:
results, names = list(), list()
for name, model in models.items():
    y_pred = evaluate_model(model, X, y)
    results.append(y_pred)
    names.append(name)
    print('>%s %.3f (%.3f)' % (name, mean(y_pred), std(y_pred)))


>stacking 0.156 (0.363)


In [53]:
print(y_pred)

[0 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 1 1 0 1 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0
 0 1 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 0 1 0 0 0 0 0 0 0 0 0
 1 0 0 0 0 1 1 0 1 0 0]


In [54]:
from sklearn.metrics import classification_report, confusion_matrix
cm=confusion_matrix(y_test,y_pred)
print(cm)

[[94 15]
 [ 9  4]]


In [55]:
# predict probabilities
from sklearn.metrics import roc_auc_score
pred_prob = model.predict_proba(X_test)
auc_score = roc_auc_score(y_test, pred_prob[:,1])
print("Area Under Curve=")
print(auc_score)

Area Under Curve=
0.5850388143966125


In [56]:
from sklearn.metrics import matthews_corrcoef
mcc=matthews_corrcoef(y_test,y_pred)
print("Matthews correlation coefficient=")
print(mcc)

Matthews correlation coefficient=
0.14472271610763604


In [57]:
from sklearn.metrics import balanced_accuracy_score
bac=balanced_accuracy_score(y_test, y_pred)
print("Balanced Accuracy Score=")
print(bac)

Balanced Accuracy Score=
0.5850388143966125


In [58]:
from imblearn.metrics import geometric_mean_score
gmean=geometric_mean_score(y_test, y_pred)
print("Geometric Mean Score=")
print(gmean)

Geometric Mean Score=
0.5151206941771151


In [59]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.91      0.86      0.89       109
           1       0.21      0.31      0.25        13

    accuracy                           0.80       122
   macro avg       0.56      0.59      0.57       122
weighted avg       0.84      0.80      0.82       122

